In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d ananthu017/squad-csv-format
!unzip squad-csv-format

In [1]:
import pandas as pd 

df = pd.read_csv('/content/SQuAD_csv.csv')
df

,Unnamed: 0,context,question,id,answer_start,text
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,269,in the late 1990s
1,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,207,singing and dancing
2,2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,526,2003
3,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,166,"Houston, Texas"
4,4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,276,late 1990s
...,...,...,...,...,...,...
86816,271,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,5735d259012e2f140011a09d,229,Oregon
86817,272,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,5735d259012e2f140011a09e,414,Rangoon
86818,273,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,5735d259012e2f140011a09f,476,Minsk
86819,274,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,5735d259012e2f140011a0a0,199,1975


In [2]:
class WordProcessing:
    def __init__(self):
        self.word2token = dict()
        self.token2word = dict()
        self.total_cnt = 0
        self.add_word("<START>")
        self.add_word("<END>")

    def add_word(self, word: str):
        if word not in self.word2token.keys():
            self.total_cnt += 1
            word_id = self.total_cnt 
            self.word2token[word] = word_id
            self.token2word[word_id] = word

    def get_token(self, word: str) -> int:
        return self.word2token[word]

    def get_word(self, token: int) -> str:
        return self.token2word[token]

    def __len__(self):
        return self.total_cnt

In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm

processor = WordProcessing()

for i in tqdm(range(df.shape[0])):
    words = word_tokenize(str(df['context'].iloc[i]))
    for word in words:
        processor.add_word(word)
    words = word_tokenize(str(df['question'].iloc[i]))
    for word in words:
        processor.add_word(word)
    words = word_tokenize(str(df['text'].iloc[i]))
    for word in words:
        processor.add_word(word)

  0%|          | 0/86821 [00:00<?, ?it/s]

In [6]:
import torch
import torch.nn as nn

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [7]:
class FCN(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_dim, 2 * (in_dim + out_dim)),
            nn.LeakyReLU(0.01),
            nn.Linear(2 * (in_dim + out_dim), out_dim)
        )

    def forward(self, input):
        return self.model(input)

In [8]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d, max_len = 5000):
        super().__init__()
        self.d = d
        self.max_len = max_len
        self._build()

    def _build(self):
        position = torch.arange(self.max_len).unsqueeze(1).to(device)
        div_term = torch.exp(torch.arange(0, self.d, 2) * (-math.log(10000.0) / self.d)).to(device)
        self.pe = torch.zeros(self.max_len, 1, self.d).to(device)
        self.pe[:, 0, 0::2] = torch.sin(position * div_term)
        self.pe[:, 0, 1::2] = torch.cos(position * div_term)    
    
    def forward(self, x):
        return x + self.pe[:x.size(0)]

In [9]:
from math import sqrt

class MultiHeadAttention(nn.Module):
    def __init__(self, dim_n, dim_m):
        super().__init__()
        self.dim_m = dim_m
        self.encoder_Q = nn.Linear(dim_n, dim_m).to(device)
        self.encoder_V = nn.Linear(dim_n, dim_m).to(device)
        self.encoder_K = nn.Linear(dim_n, dim_m).to(device)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        Q = self.encoder_Q(X.to(device))
        K = self.encoder_K(X.to(device))
        V = self.encoder_V(X.to(device))
        attention = Q @ torch.transpose(K, 1, 2) # n * m * m * n = n * n
        attention /= sqrt(self.dim_m)
        return self.softmax(attention) @ V

In [10]:
class MultiHeadAttentionDecoder(nn.Module):
    def __init__(self, dim_n, dim_m):
        super().__init__()
        self.dim_m = dim_m
        self.encoder_Q = nn.Linear(dim_n, dim_m).to(device)
        self.softmax = nn.Softmax(dim=1).to(device)

    def forward(self, X, K, V):
        Q = self.encoder_Q(X)
        attention = Q @ torch.transpose(K, 1, 2) # n * m * m * n = n * n
        attention /= sqrt(self.dim_m)
        return self.softmax(attention) @ V        

In [11]:
class Encoder(nn.Module):
    def __init__(self, vocab_size,
                 num_attentions = 4,
                 embedding_dim = 100,
                 ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.positional_encoding = PositionalEncoding(embedding_dim)
        self.attention_layers = []
        self.fc_layers = []
        self.layer_norms = []
        for i in range(num_attentions):
            self.attention_layers.append(MultiHeadAttention(embedding_dim, embedding_dim))
            self.fc_layers.append(nn.Sequential(
                    nn.Linear(embedding_dim, 4 * embedding_dim),
                    nn.ReLU(),
                    nn.Linear(4 * embedding_dim, embedding_dim)
                ).to(device)
            )
            self.layer_norms.append(nn.LayerNorm(embedding_dim).to(device))
        self.num_attensions = num_attentions

    def forward(self, X):
        embeddings = self.embedding(X)
        embeddings = embeddings.permute(1, 0, 2)
        hidden = self.positional_encoding(embeddings)
        hiddens = [hidden]
        for i in range(self.num_attensions):
            hiddens.append(hiddens[-1] + self.attention_layers[i](hiddens[-1]))
            hiddens.append(hiddens[-1] + self.fc_layers[i](hiddens[-1]))
            hiddens.append(self.layer_norms[i](hiddens[-1]))
        return hiddens[-1]

In [12]:
class Decoder(nn.Module):
    def __init__(self,
                 vocab_size,
                 num_attentions = 4,
                 embedding_dim = 100):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.positional_encoding = PositionalEncoding(embedding_dim)
        self.attention_layers = []
        self.fc_layers = []
        self.layer_norms = []
        self.decoder_attention = []
        for i in range(num_attentions):
            self.attention_layers.append(MultiHeadAttention(embedding_dim, embedding_dim))
            self.fc_layers.append(nn.Sequential(
                    nn.Linear(embedding_dim, 4 * embedding_dim),
                    nn.ReLU(),
                    nn.Linear(4 * embedding_dim, embedding_dim)
                ).to(device))
            self.layer_norms.append(nn.LayerNorm(embedding_dim).to(device))
        self.decoder_attention.append(MultiHeadAttentionDecoder(embedding_dim, embedding_dim))
        self.num_attensions = num_attentions

    def forward(self, X, encoder_output_K, encoder_output_V):
        embeddings1 = self.embedding(X)
        embeddings = embeddings1.permute(1, 0, 2)
        hidden = self.positional_encoding(embeddings)
        hiddens = [hidden]
        hiddens.append(hiddens[-1] + self.decoder_attention[0](X = hiddens[-1], K = encoder_output_K, V = encoder_output_K))

        for i in range(self.num_attensions):
            hiddens.append(hiddens[-1] + self.attention_layers[i](hiddens[-1]))
            hiddens.append(hiddens[-1] + self.fc_layers[i](hiddens[-1]))
            hiddens.append(self.layer_norms[i](hiddens[-1]))
        last = hiddens[-1]
        return last

In [13]:
vocab_size = len(processor) + 100

In [14]:
vocab_size

115699

In [15]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, 
                 word_processor,
                 df,
                 max_length = 700,
                 context_column = "context",
                 question_column = "question",
                 answer_column = "text"):
        self.word_processor = word_processor 
        self.df = df 
        self.context_column = context_column
        self.question_column = question_column
        self.answer_column = answer_column
        self.max_length = max_length

    def __getitem__(self, index):
        context_text = word_tokenize(str(df[self.context_column].iloc[index]))
        question_text = word_tokenize(str(df[self.question_column].iloc[index]))
        answer_text = word_tokenize(str(df[self.answer_column].iloc[index]))
        
        context_tensor = [self.word_processor.get_token(word) for word in context_text]
        question_tensor = [self.word_processor.get_token(word) for word in question_text]
        answer_tensor = [self.word_processor.get_token(word) for word in answer_text]

        context_tensor += [0] * (self.max_length - len(context_tensor))
        question_tensor += [0] * (self.max_length - len(question_tensor))
        answer_tensor += [0] * (self.max_length - len(answer_tensor))

        context_tensor = torch.LongTensor(context_tensor).to(device)
        question_tensor = torch.LongTensor(question_tensor).to(device)
        answer_tensor = torch.LongTensor(answer_tensor).to(device)

        return {'context': context_tensor.to(device),
                'question': question_tensor.to(device),
                'answer': answer_tensor.to(device)}

    def __len__(self):
        return self.df.shape[0]

In [16]:
from torch.utils.data import random_split

dataset = TextDataset(processor, df)

train_len = int(0.7 * len(dataset)) 
val_len = len(dataset) - train_len 

dataset_train, dataset_val = random_split(dataset, [train_len, val_len])

loader_train = DataLoader(dataset_train, batch_size = 10, num_workers = 0)
loader_val = DataLoader(dataset_val, batch_size = 10, num_workers = 0)

In [17]:
embedding_dim = 100

encoder_context = Encoder(vocab_size = vocab_size, embedding_dim=embedding_dim).to(device)
encoder_questions = Encoder(vocab_size = vocab_size, embedding_dim=embedding_dim).to(device)
decoder_answers = Decoder(vocab_size = vocab_size, embedding_dim=embedding_dim).to(device)

net2k = nn.Sequential(nn.Linear(2 * embedding_dim, embedding_dim),
                      nn.ReLU(),
                      nn.Linear(embedding_dim, embedding_dim)).to(device)

net2v = nn.Sequential(nn.Linear(2 * embedding_dim, embedding_dim),
                      nn.ReLU(),
                      nn.Linear(embedding_dim, embedding_dim)).to(device)

last = nn.Sequential(nn.Linear(embedding_dim, vocab_size),
                     nn.LayerNorm(vocab_size),
                     nn.Softmax()).to(device)

In [18]:
learning_rate = 6e-4

optimizer_context = torch.optim.Adam(encoder_context.parameters(), lr = learning_rate)
optimizer_questions = torch.optim.Adam(encoder_questions.parameters(), lr = learning_rate)
optimizer_answers = torch.optim.Adam(decoder_answers.parameters(), lr = learning_rate)
optimizer_net2k = torch.optim.Adam(net2k.parameters(), lr = learning_rate)
optimizer_net2v = torch.optim.Adam(net2v.parameters(), lr = learning_rate)
optimizer_last = torch.optim.Adam(last.parameters(), lr = learning_rate)

In [19]:
def zero_grad():
    optimizer_context.zero_grad()
    optimizer_questions.zero_grad()
    optimizer_answers.zero_grad()
    optimizer_net2k.zero_grad()
    optimizer_net2v.zero_grad()
    optimizer_last.zero_grad()

def step():
    optimizer_context.step()
    optimizer_questions.step()
    optimizer_answers.step()
    optimizer_net2k.step()
    optimizer_net2v.step()
    optimizer_last.step()

In [ ]:
from tqdm.auto import tqdm

num_epochs = 1
criterion = nn.CrossEntropyLoss()
torch.autograd.set_detect_anomaly(True)
for epoch in tqdm(range(num_epochs)):
    for batch in tqdm(loader_train):
        encoded_context = encoder_context(batch['context'].permute(1, 0).to(device))
        encoded_question = encoder_questions(batch['context'].permute(1, 0).to(device))
        concated = torch.cat([encoded_context, encoded_question], 2)
        K = net2k(concated)
        V = net2v(concated)
        start_embedding = [torch.ones((batch['answer'].shape[0], 1)).to(device).to(torch.long)]
        zero_grad()

        loss = 0
        
        for i in range(batch['answer'].shape[1]):
            if batch['answer'][:, i].reshape(-1).max().item() == 0:
                break
            loss += criterion(last(decoder_answers(start_embedding[-1].reshape(1, 10), K, V)).squeeze(1),
                             batch['answer'][:, i].reshape(batch['answer'].shape[0]))
            start_embedding.append(batch['answer'][:, i].reshape(-1, 1))
        loss.backward(retain_graph=True)

        step()
    sum_loss, cnt_loss = 0, 0
    for batch in tqdm(loader_val):
        encoded_context = encoder_context(batch['context'].permute(1, 0))
        encoded_question = encoder_questions(batch['context'].permute(1, 0))
        concated = torch.cat([encoded_context, encoded_question], 2)
        K = net2k(concated)
        V = net2v(concated)
        start_embedding = [torch.ones((batch['answer'].shape[0], 1)).to(device).to(torch.long)]

        loss = 0
        for i in range(batch['answer'].shape[1]):
            if batch['answer'][:, i].reshape(-1).max().item() == 0:
                break
            loss += criterion(last(decoder_answers(start_embedding[-1].reshape(1, 10), K, V)).squeeze(1),
                             batch['answer'][:, i].reshape(batch['answer'].shape[0]))
            start_embedding.append(batch['answer'][:, i].reshape(-1, 1))
        sum_loss += loss.item()
        cnt_loss += 1
    print(sum_loss / cnt_loss)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6078 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
